In [1]:
import requests,time,datetime
import json
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lxml import etree
import xlwt
import xlrd
import utils.DateAndTimeUtil as datu
import utils.CommonUtil as cu

In [2]:
df_hsA = pd.read_excel("data/沪深A股.xls",converters= {u'StockCode':str})

In [34]:
df = df_hsA

In [48]:
#获取年报
def getAnnualReport (stock_code,date):
    #拼接完整股票代码
    symbol= "SZ"+stock_code if stock_code[0] in ["0", "3"] else "SH"+stock_code
    # 报告数量取1，会从制定日期向前获取最近的一个报告
    count = 1

    # https://stock.xueqiu.com/v5/stock/finance/cn/indicator.json?symbol=SH601318&type=all&is_detail=true&count=5&timestamp=1574826156123
    url = "https://stock.xueqiu.com/v5/stock/finance/cn/indicator.json?" \
            "symbol={}&type=all&is_detail=true&count={}&timestamp={}" .\
            format(symbol, count,convertDate2TimeStamp(date))

    # header里面必须加入Cookie，否则会报400错误
    headers = {"user-agent":"PostmanRuntime/7.13.0",
               "Cookie":"xq_a_token.sig=PsMGyjjYlfq-rIaaHcjsTYLCLUI; xq_r_token.sig=ZjX7QUQXP0ego37J0qiLOHGZH7Y; device_id=0887667ae157d8aecf875f314dcb8289; s=cn151gduqd; Hm_lvt_1db88642e346389874251b5a1eded6e3=1574730685; remember=1; xq_a_token=285dde3cce95b000682556142a7dcb0e8074ef3f; xqat=285dde3cce95b000682556142a7dcb0e8074ef3f; xq_r_token=91df29cf76abbdd617283a2178b3369d87d7be26; xq_is_login=1; u=1333010681; Hm_lpvt_1db88642e346389874251b5a1eded6e3=1574752875"}

    data = requests.get(url, headers = headers)
    try:
        data = pd.read_json(data.text, dtype=False, orient='records')
    except Exception as e:
        return {}
    # title = data['data']['quote_name'] + '({})'.format(symbol)
    try:
        dict_fin = pd.Series(data['data']['list'])[0]
        return dict_fin
    except Exception as e:
        return {}

# 获取历史动态市盈率
def getPETTM(stock_code,years = 5):
    url = 'http://www.dashiyetouzi.com/tools/compare/historical_valuation_data.php'
    # 这里必须带上Cookie，否则获取不到数据
    headers = {"user-agent":"PostmanRuntime/7.13.0",
              "Cookie":"PHPSESSID=3n0ka6t15s4prls0bd7mll7u30; Hm_lvt_210e7fd46c913658d1ca5581797c34e3=1574836678; stock=%u98DE%u79D1%u7535%u5668%3BSH%3B603868; Hm_lpvt_210e7fd46c913658d1ca5581797c34e3=1574918719"}
    
    from_date = datu.timeStamp2Date(time.time() - (datu.oneDaySecond() * years * 365))
    to_date = datu.timeStamp2Date(time.time())
    params = (('report_type', 'pettm'),('report_stock_id', stock_code),('from_date', from_date),('to_date',to_date ))
    response = requests.post(url, headers = headers,data=params)
    
    return json.loads(response.text)

#计算市盈率中位数
def getPEMedian(stock_code):
    data = getPETTM(stock_code)
    pe_list = []
    try:
        for pair in data['list']:
            pe_list.append(pair[1])
        pe_median = np.median(pe_list)
        pe_newest = pe_list[len(pe_list) - 1]
    except Exception as e:
        return 0
    
    return pe_median

In [73]:
years = {'2013':'2013-12-31','2014':'2014-12-31','2015':'2015-12-31','2016':'2016-12-31','2017':'2017-12-31','2018':'2018-12-31'}

In [74]:
workbook = xlwt.Workbook()

year = '2013'
# stock_code：股票代码
# stock_name：股票简称
'-------财务数据-------'
# total_revenue：营业收入
# net_profit_atsopc：净利润

# basic_eps：每股收益
# np_per_share：每股净资产
# operate_cash_flow_ps：现金流

# avg_roe：净资产收益率
asset_liab_ratio：资产负债率

heads = ['stock_code','stock_name','total_revenue','total_revenue_gr','net_profit_atsopc','net_profit_atsopc_gr','basic_eps','basic_eps_gr',
         'np_per_share','np_per_share_gr','operate_cash_flow_ps','operate_cash_flow_ps_gr','avg_roe','asset_liab_ratio']
#添加表头
sheet = workbook.add_sheet(year + "年报") 
for h in range(len(heads)):
    sheet.write(0, h, heads[h]) 

row_index = 0
for index, row in df.iterrows():
    stock_code = row['StockCode']
    dict_ar = getAnnualReport(stock_code,years[year])
    if len(dict_ar) == 0:
        continue
    if year not in dict_ar['report_name']:
        continue
    row_index = row_index + 1
    cu.printProgress('获取第'+str(row_index)+'条')
    
    sheet.write(row_index,0,stock_code)
    sheet.write(row_index,1,row['StockName'])
    sheet.write(row_index,2,dict_ar['total_revenue'][0])
    sheet.write(row_index,3,dict_ar['total_revenue'][1])
    sheet.write(row_index,4,dict_ar['net_profit_atsopc'][0])
    sheet.write(row_index,5,dict_ar['net_profit_atsopc'][1])
    sheet.write(row_index,6,dict_ar['basic_eps'][0])
    sheet.write(row_index,7,dict_ar['basic_eps'][1])
    sheet.write(row_index,8,dict_ar['np_per_share'][0])
    sheet.write(row_index,9,dict_ar['np_per_share'][1])
    sheet.write(row_index,10,dict_ar['operate_cash_flow_ps'][0])
    sheet.write(row_index,11,dict_ar['operate_cash_flow_ps'][1])
    sheet.write(row_index,12,float(0 if dict_ar['avg_roe'][0] is None else dict_ar['avg_roe'][0])/100)
    sheet.write(row_index,13,float(dict_ar['asset_liab_ratio'][0])/100)
    
#     if index == 10:
#         break

dir_path = 'E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/'
workbook.save(dir_path + year + '年报.xls')

获取第3599条

In [5]:
# 显示所有列
pd.set_option('display.max_columns', None)
# 显示所有行
pd.set_option('display.max_rows', None)

In [49]:
df_2014 = pd.read_excel("data/2014年报.xls",converters= {u'stock_code':str})
df_2015 = pd.read_excel("data/2015年报.xls",converters= {u'stock_code':str})
df_2016 = pd.read_excel("data/2016年报.xls",converters= {u'stock_code':str})
df_2017 = pd.read_excel("data/2017年报.xls",converters= {u'stock_code':str})
df_2018 = pd.read_excel("data/2018年报.xls",converters= {u'stock_code':str})

In [50]:
code_list = []
reports = [df_2014,df_2015,df_2016,df_2017,df_2018]
for df_report in reports:
    codes = []
    for index, row in df_report.iterrows():
        # 净资产收益率大于15%
        if (row['stock_code'][0] != '3') & \
            (row['avg_roe'] > 0.15) & \
           (row['total_revenue_gr'] > 0) & \
            (row['basic_eps'] > 0) & \
            (row['np_per_share'] > 0):
#             (row['operate_cash_flow_ps'] > 0) & \
#             (row['operate_cash_flow_ps_gr'] > 0):
            #满足条件的加入
            codes.append(row['stock_code'])
            
    code_list.append(codes)

In [51]:
intersection = code_list[0]
# 取每年的交集
for i in range(1,len(code_list)):
    intersection = list(set(intersection).intersection(set(code_list[i])))

In [52]:
df_target = df[df['StockCode'].apply(lambda x: (x in intersection))]

In [53]:
df_target.shape

(101, 19)

In [55]:
df_target = df_target[df_target['PE_D'].apply(lambda x:float(x) != 0)]
df_target = df_target[df_target['PE_S'].apply(lambda x:float(x) != 0)]
# 新增市盈率中位数
df_target['PE_M'] = df_target['StockCode'].apply(lambda x:getPEMedian(x))
df_target = df_target[df_target['PE_M'] != 0]

df_target['pe_rate'] = df_target.apply(lambda x:(float(x['PE_D']) / x['PE_M']),axis=1)
df_target = df_target[df_target['pe_rate'] < 1]

In [56]:
df_target.shape

(65, 21)

In [57]:
df_target.sort_values(by=['pe_rate'],ascending=True)

,StockCode,StockName,Price,PriceLimit,QuantityRelativeR,TurnoverRate,Trade,PB,PE_S,PE_D,EarningsPerShare,NetProfitDes,ROE,GrossProfitRate,NetAssetValuePerShare,CapitalStock,ScaleShareType,FinanceAnalize,MoneyFlowPerShare,PE_M,pe_rate
2773,600566,济川药业,22.52,-0.66%,0.70,0.35%,医药生物 — 中药Ⅱ,3.24,10.95,10.613,1.60元,13.06亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降0.24%\t...,23.05%,84.45%,7.00元,8.15亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,"三线蓝筹,白马股,绩优股,",2.19元,27.670,0.383556
3013,002262,恩华药业,11.00,-0.99%,0.91,0.69%,医药生物 — 化学制药,3.26,21.58,16.311,0.52元,5.21亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长23.34%\t...,16.16%,60.53%,3.41元,10.19亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,"绩优股,",0.55元,41.670,0.391433
591,603579,荣泰健康,27.83,1.16%,0.63,0.95%,家用电器 — 白色家电,2.46,15.45,13.824,1.50元,2.09亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长13.69%\t...,13.44%,29.42%,11.17元,1.40亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t小盘股\t\t\t\t\t\t\t\...,"绩优股,",2.16元,33.030,0.418529
2143,002572,索菲亚,17.52,-0.06%,0.69,0.81%,轻工制造 — 家用轻工,3.15,16.68,16.664,0.79元,7.20亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长4.05%\t\...,14.80%,37.25%,5.57元,9.12亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,"绩优股,",0.66元,39.520,0.421660
1591,002174,游族网络,15.63,0.26%,0.86,2.64%,传媒 — 互联网传媒,2.56,13.47,14.456,0.81元,7.05亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长3.29%\t\...,14.14%,53.90%,5.97元,8.88亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,"绩优股,",0.18元,30.950,0.467076
2388,600511,国药股份,24.43,-0.29%,0.53,0.73%,医药生物 — 医药商业Ⅱ,1.87,13.16,12.941,1.40元,10.71亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长12.71%\...,11.16%,8.27%,13.08元,7.55亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,"白马股,",1.42元,26.880,0.481436
1874,600340,华夏幸福,27.93,0.04%,0.60,0.17%,房地产 — 房地产开发Ⅱ,2.12,7.18,6.494,3.18元,97.46亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长23.71%\...,25.19%,40.40%,13.21元,30.14亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,"二线蓝筹,白马股,绩优股,",-13.18元,13.420,0.483905
474,600535,天士力,15.26,1.40%,2.55,1.52%,医药生物 — 中药Ⅱ,2.05,14.73,15.083,0.75元,11.32亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降17.31%\...,10.39%,31.38%,7.35元,15.13亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,"三线蓝筹,绩优股,",0.83元,29.940,0.503774
3161,000963,华东医药,24.60,-1.24%,1.37,0.72%,医药生物 — 化学制药,3.74,19.23,14.771,1.26元,22.13亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长22.33%\...,20.14%,32.34%,6.66元,17.50亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,"二线蓝筹,",0.52元,28.685,0.514938
362,603886,元祖股份,18.44,1.65%,0.68,2.01%,食品饮料 — 食品加工,2.87,18.08,11.899,1.15元,2.75亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长6.94%\t\...,18.56%,65.91%,6.33元,2.40亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t小盘股\t\t\t\t\t\t\t\...,"绩优股,",2.64元,22.810,0.521657


In [46]:
df_target.shape

(95, 21)